In [2]:
import sim          # Library for coppelia communication       
import sympy as sp  # Library to work with symbolic variables

In [3]:
# Function to calculate symbolic denavit transformations from parameters

def symTfromDH(theta, d, a, alpha):

    Rz = sp.Matrix([[sp.cos(theta), -sp.sin(theta), 0, 0],
                   [sp.sin(theta), sp.cos(theta), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    tz = sp.Matrix([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, d],
                   [0, 0, 0, 1]])
    ta = sp.Matrix([[1, 0, 0, a],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    Rx = sp.Matrix([[1, 0, 0, 0],
                   [0, sp.cos(alpha), -sp.sin(alpha), 0],
                   [0, sp.sin(alpha), sp.cos(alpha), 0],
                   [0, 0, 0, 1]])
    T = Rz*tz*ta*Rx
    return T

## Calculate Denavit Transformations

In [4]:
# Robot dimensions
L3 = 150
L4 = 100
L5 = 50

q1 = sp.symbols('q1') # First rotational joint angle
q2 = sp.symbols('q2') # Second rotational joint angle
l1 = sp.symbols('l1') # First prismatic joint length
l2 = sp.symbols('l2') # Second prismatic joint length

# Calculated denavit parameters
theta = [sp.pi/2,-sp.pi/2, 0 , q1 ,q2]
d = [L3,l1,l2,0,0]
a = [0,0,0,L4,L5]
alpha = [sp.pi/2,-sp.pi/2,0,0,0]

# Partial transformations to end effector
T01 = symTfromDH(theta[0],d[0],a[0],alpha[0])
T12 = symTfromDH(theta[1],d[1],a[1],alpha[1])
T23 = symTfromDH(theta[2],d[2],a[2],alpha[2])
T34 = symTfromDH(theta[3],d[3],a[3],alpha[3])
T45 = symTfromDH(theta[4],d[4],a[4],alpha[4])

# Final tranformation
T05 = T01*T12*T23*T34*T45
T05

Matrix([
[-sin(q1)*cos(q2) - sin(q2)*cos(q1), sin(q1)*sin(q2) - cos(q1)*cos(q2), 0,  l1 - 50*sin(q1)*cos(q2) - 100*sin(q1) - 50*sin(q2)*cos(q1)],
[                                 0,                                 0, 1,                                                          l2],
[ sin(q1)*sin(q2) - cos(q1)*cos(q2), sin(q1)*cos(q2) + sin(q2)*cos(q1), 0, 50*sin(q1)*sin(q2) - 50*cos(q1)*cos(q2) - 100*cos(q1) + 150],
[                                 0,                                 0, 0,                                                           1]])

## Direct Kinematics

In [50]:
# Evaluate end effector transformation

values ={      # Limits
q1:0,           # (0-2pi)
q2:sp.pi/2,     # (-pi/2,pi/2)
l1:0,           # (0,300cm)
l2:0           # (0,200cm)
}

T05.evalf(subs=values)

Matrix([
[   -1.0, 0.e-678,   0, -50.0],
[      0,       0, 1.0,     0],
[0.e-678,     1.0,   0,  50.0],
[      0,       0,   0,   1.0]])

In [3]:
# Function to connect to coppelia simulator

def connect(port):
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5)
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID

In [6]:
# Connection to coppelia 
clientID = connect(25000)
# Get joints and end effector handlers
retCode,pjoint0=sim.simxGetObjectHandle(clientID,'pjoint0',sim.simx_opmode_blocking) 
retCode,pjoint1=sim.simxGetObjectHandle(clientID,'pjoint1',sim.simx_opmode_blocking) 
retCode,rjoint0=sim.simxGetObjectHandle(clientID,'rjoint0',sim.simx_opmode_blocking) 
retCode,rjoint1=sim.simxGetObjectHandle(clientID,'rjoint1',sim.simx_opmode_blocking) 
retCode,end_effector=sim.simxGetObjectHandle(clientID,'EndEffector',sim.simx_opmode_blocking) 

# Read end effector actual position
returnCode,ef_pos = sim.simxGetObjectPosition(clientID,end_effector,-1,sim.simx_opmode_blocking) # Read tip position relative to base(-1) in meters
print([ef_pos])

conectado a 25000
[[1.49540376663208, 0.9999954700469971, 0.4977395534515381]]


In [4]:
# Send new values for joints
l1 = 0.5
l2 = 0.5
q1 = -45*sp.pi/180
q2 = -40*sp.pi/180
retCode = sim.simxSetJointTargetPosition(clientID,pjoint0,l1, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID,pjoint1,l2, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID,rjoint0,q1, sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID,rjoint1,q2, sim.simx_opmode_oneshot)
print(retCode)

# Read new end effector position
returnCode,ef_pos = sim.simxGetObjectPosition(clientID,end_effector,-1,sim.simx_opmode_blocking) # Read tip position relative to base(-1) in meters
print([ef_pos])

NameError: name 'clientID' is not defined

## Inverse Kinematics

In [5]:
T05

Matrix([
[-sin(q1)*cos(q2) - sin(q2)*cos(q1), sin(q1)*sin(q2) - cos(q1)*cos(q2), 0,  l1 - 50*sin(q1)*cos(q2) - 100*sin(q1) - 50*sin(q2)*cos(q1)],
[                                 0,                                 0, 1,                                                          l2],
[ sin(q1)*sin(q2) - cos(q1)*cos(q2), sin(q1)*cos(q2) + sin(q2)*cos(q1), 0, 50*sin(q1)*sin(q2) - 50*cos(q1)*cos(q2) - 100*cos(q1) + 150],
[                                 0,                                 0, 0,                                                           1]])

In [24]:
# Define end effector postition
x = 150
y = 150
z = 20

eqx = T05[3] - x
eqy = T05[7] - y
eqz = T05[11] - z
eq_or = q1+q2 # orientation of end effector 


eq = (eqx,eqy,eqz,eq_or)
print(eq)

(l1 - 50*sin(q1)*cos(q2) - 100*sin(q1) - 50*sin(q2)*cos(q1) - 150, l2 - 150, 50*sin(q1)*sin(q2) - 50*cos(q1)*cos(q2) - 100*cos(q1) + 150, q1 + q2)


In [18]:
eqy

l2

In [25]:
# calculamos la cinemática inversa
print("calculando la cinemática inversa...")
q = sp.nsolve(eq, (q1,q2,l1,l2), (1,1,1,1), prec=4)
print(q)



calculando la cinemática inversa...
Matrix([[0.001280], [-0.001280], [150.1], [150.0]])


In [ ]:
# Move robot to newly claculated position

retCode = sim.simxSetJointTargetPosition(clientID, rjoint1,q[0], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, rjoint0,q[1], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, pjoint0,q[2], sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetPosition(clientID, pjoint1,q[3], sim.simx_opmode_blocking)